# FIRST DOWNLOAD DATA FOLDER

lets use 5/12 as a demo

go to https://nuwildcat.sharepoint.com/:f:/r/sites/FSM-TreschLab/Shared%20Documents/General/Cortical/Data_Folders/rat-annie/5-12?csf=1&web=1&e=KLIIX5 and download

place into rat-fes/data/here

In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

from src.folder_handler import *
from src.cort_processor import *
from src.tdt_support import *
from src.plotter import *
from src.decoders import *
from src.dummy import *
import pickle

# the next section is loading data through Jupyter. I don't recommend this method. Skip to loading data into pickle if you want to do that way

## LOADING DATA THROUGH JUPYTER
data should be a single folder. Inside there should be a 'angles' (with angles), 'pose-3d', and 'tdt', and config.yaml folders named exactly that

In [ ]:
session512 = CortProcessor('/rat-fes/data/5-12') #insert name of your folder here

## process data
here, we crop our data, turn our raw data into rates, and we resample our angles to be same times as binned spikes

In [ ]:
rates, kins = session512.process()

### now our rates are stored under session512.data['rates'], our angles are stored under session512.data['angles'], and our coords are stored under session512.data['coords']

In [ ]:
print(session512.data['rates'][0].shape)
print(session512.data['angles'][0].shape)
print(session512.data['coords'][0].shape)

# recommended way to load data: using pickles (and process_cort.py in scripts)

in order to process data using command line, navigate to /scripts, and run ``` python3 process_cort.py 'path_to_data_folder' ```

this will generate a file in data/pickles/______ 

then you can load the pickled CortProcessor object in the following line of code

In [3]:
with open('/home/diya/Documents/rat-fes/data/pickles/rat-annie_5-12_session.pkl', 'rb') as inp: #YOUR PATH HERE
    session512 = pickle.load(inp)

In [4]:
print(session512.data['rates'][0].shape)
print(session512.data['angles'][0].shape)
print(session512.data['coords'][0].shape)

(2962, 32)
(2962, 7)
(2962, 17, 3)


# pipeline processing, like getting toe height

In [ ]:
toe_height = session512.process_toe_height()

In [ ]:
print(session512.data['toe_height'][0].shape)

# and now some decoding

## angles

In [5]:
session512.decode_angles?

Signature: session512.decode_angles(X=None, Y=None)
Docstring:
takes list of rates, angles, then using a wiener filter to decode. 
if no parameters are passed, uses data['rates'] and data['angles']

returns best_h, vaf (array of all angles and all folds), test_x (the
x test set that had best performance, and test_y (the y test set that
had best formance)
File:      ~/Documents/rat-fes/src/cort_processor.py
Type:      method


In [10]:
h_angles, vaf_angles, test_x_angles, test_y_angles = session512.decode_angles()

In [12]:
np.average(vaf_angles, 1)

array([0.09069322, 0.27032082, 0.15030921, 0.23722803, 0.16953658,
       0.18049855, 0.19855415])

## toe_height

In [ ]:
h_toe, vaf_toe, best_x_toe, best_y_toe = session512.decode_toe_height()

In [ ]:
average_toe_vaf = np.average(vaf_toe)
print(average_toe_vaf)

In [ ]:
plot_wiener_filter_predic(best_x_toe, best_y_toe, h_toe)

# or dividing into gaits, then plotting sorted raster.

In [ ]:
session512.get_gait_indices?

In [ ]:
gait_indices, avg_gait_samples = session512.get_gait_indices()

In [ ]:
X_gait, Y_gait = session512.divide_into_gaits()

In [ ]:
df = session512.neuron_tuning()

In [ ]:
plot_raster(df)